In [1]:
import os
from os import path
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import glob
import numpy as np
import torch 
from transformers import ViTImageProcessor
from PIL import Image
from transformers import ViTForImageClassification
from torchvision.transforms import functional as F
import natsort
import random

2023-08-04 14:42:46.199724: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
PERFORMANCE_RESULTS = pd.DataFrame(columns=['Model', 'EvalSet', 'Accuracy', 
                       'M-Precision', 'M-Recall', 
                       'M-F1-Score', 'W-Precision',
                       'W-Recall', 'W-F1-Score'])

# for KK in list([7,10,18,22,55,77]): 
KK=22

random.seed(KK)
np.random.seed(KK)
torch.manual_seed(KK)
tf.random.set_seed(KK)



img_IDs = []
updated_pathnames = []
dir_path = os.path.join(os.getcwd() , 'TRAINING')
img_filenames= natsort.natsorted(os.listdir(dir_path))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(img_filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
        path_name=os.path.join(dir_path, filename)
        im = Image.open(path_name)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
            continue
        img_IDs.append(filename)
        updated_pathnames.append(path_name)  #gives 9986 results


def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))


txt_file = pd.read_csv('./training1.csv', delimiter='\t')
sub_txt_file = txt_file[['file_name', 'misogynous']]
sub_txt_file['prefix_file_name'] = sub_txt_file['file_name'].str.extract('(\d+)').astype(int)
sub_txt_file = sub_txt_file.sort_values(by='prefix_file_name', ascending=True)


txt_IDs = list(sub_txt_file.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(txt_IDs, img_IDs)
print('len of common strings', len(common_strings))

common_IDs = natsort.natsorted(common_strings)
print(len(common_IDs), len(txt_IDs), len(img_IDs))


filtered_txt_file = sub_txt_file[sub_txt_file['file_name'].isin(common_IDs)].reset_index(drop=True)
filtered_txt_file

train_labels= list(filtered_txt_file['misogynous'])


train_images_path = []
for i in common_IDs:
    path_name=os.path.join(dir_path, i)
    train_images_path.append(path_name)



######################################## TEST IMAGES PATH ###################################
# Get IDs from test text data
test1 = pd.read_csv('Test.csv', delimiter='\t')
test_labels = pd.read_csv('test_labels.txt', 
                          delimiter='\t',
                         header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

merged_test = pd.merge(test1, test_labels, on='file_name', how='inner')

# Sort the dataframe with natural ordering of the IDs
merged_test['prefix_file_name'] = merged_test['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
merged_test1 = merged_test.sort_values(by='prefix_file_name', ascending=True)
merged_test1

# # train = train.rename(columns={'Text Transcription': 'text'})
test2 = merged_test1.rename(columns={'Text Transcription': 'text'})
test_txt_IDs = list(test2['file_name'])



# GET IDs from test images data
test_dir_path = os.path.join(os.getcwd() , 'TEST')
test_img_IDs = natsort.natsorted(os.listdir(test_dir_path))  

# Get COmmon IDS
test_common_strings = get_common_strings(test_txt_IDs, test_img_IDs)
test_common_IDs = natsort.natsorted(test_common_strings)
print('len of common strings', len(test_common_IDs))


test_images_path = []
for i in test_common_IDs:
    path_name=os.path.join(test_dir_path, i)
    test_images_path.append(path_name)

# test_images_path
tst_filtered_txt_file = test2[test2['file_name'].isin(test_common_IDs)].reset_index(drop=True)
test_labels= list(tst_filtered_txt_file['misogynous'])
# test_images_path[-1], test_labels[-1]

print(len(test_images_path), len(test_labels))


# # total_img_info =train_img_info(train_images_path, train_labels)

def img_info(images_path, labels, batch_size=1):
    for i in range(0, len(images_path), batch_size):
#         image_batch = []
        for j in range(i, min(i+batch_size, len(images_path))):
            image_path = images_path[j]
            image = Image.open(image_path)
            image_info = { 'image': image, 'image_file_path': image_path, 'labels': labels[j] }
#             image_batch.append(image_info)

        yield image_info

train_images_path1 = train_images_path[0:3000]
train_labels1 = train_labels[0:3000]


total_img_info1=[]
for k in img_info(train_images_path1, train_labels1, batch_size=1):
    total_img_info1.append(k)


print('total image info created')    

import pickle
# Save the list as a pickle object
with open('15_total_img_info1.pickle', 'wb') as f:
    pickle.dump(total_img_info1, f)

print("Results saved as pickle object.")

del total_img_info1,  train_images_path1 , train_labels1

###########################################################


train_images_path2 = train_images_path[3000:6000]
train_labels2 = train_labels[3000:6000]


total_img_info2=[]
for k in img_info(train_images_path2, train_labels2, batch_size=1):
    total_img_info2.append(k)



print('total image info created')    

import pickle
# Save the list as a pickle object
with open('15_total_img_info2.pickle', 'wb') as f:
    pickle.dump(total_img_info2, f)

print("Results saved as pickle object.")

del total_img_info2, train_images_path2 , train_labels2 




########################################################

train_images_path3 = train_images_path[6000:len(train_images_path)]
train_labels3 = train_labels[6000:len(train_images_path)]


total_img_info3=[]
for k in img_info(train_images_path3, train_labels3, batch_size=1):
    total_img_info3.append(k)





print('total image info created')    

import pickle
# Save the list as a pickle object
with open('15_total_img_info3.pickle', 'wb') as f:
    pickle.dump(total_img_info3, f)

print("Results saved as pickle object.")


del total_img_info3, train_images_path3 , train_labels3 


import pickle

# List of pickle file names you want to load
pickle_files = ['15_total_img_info1.pickle', '15_total_img_info2.pickle', '15_total_img_info3.pickle']

# Create an empty list to store the combined results
total_img_info = []

# Load each pickle file and extend the combined_results list
for file_name in pickle_files:
    fn=os.path.join(os.getcwd(), file_name)
    with open(fn, 'rb') as f:
        loaded_list = pickle.load(f)
        total_img_info.extend(loaded_list)


for k in img_info(test_images_path, test_labels, batch_size=1):
    total_img_info.append(k)



/tmp/ipykernel_30007/1435002706.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_txt_file['prefix_file_name'] = sub_txt_file['file_name'].str.extract('(\d+)').astype(int)


len of common strings 9986
9986 10000 9986
len of common strings 1000
1000 1000
total image info created
Results saved as pickle object.
total image info created
Results saved as pickle object.
total image info created
Results saved as pickle object.


In [ ]:
modelpath = "google/vit-base-patch16-224-in21k"

feature_extractor =  ViTImageProcessor.from_pretrained(modelpath)

# Convert the image_info to a tensor dictionary
def convert_to_tensor(image_info):
    image = image_info['image']
    image = F.resize(image, (224, 224))  # Resize the image to the desired shape
    pixel_values = feature_extractor(images=image, return_tensors='pt')['pixel_values']
    image_info['pixel_values'] = pixel_values.squeeze()  # Remove the batch dimension
    return image_info

total_image_info_tf = [convert_to_tensor(image_info) for image_info in total_img_info]


# Convert data_dict to tensor dataset (torch.utils.data.Dataset)
data_dict = {
    'pixel_values': torch.stack([image_info['pixel_values'] for image_info in total_image_info_tf]),
    'labels': torch.tensor([image_info['labels'] for image_info in total_image_info_tf])
}



print(data_dict['pixel_values'].shape, data_dict['labels'].shape)


labels = ['non_misog', 'misog']

print('Model Configuration')
model = ViTForImageClassification.from_pretrained(
    modelpath,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

#     print(model)

import torch
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, data_dict):
        self.pixel_values = data_dict['pixel_values']
        self.labels = data_dict['labels']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get the pixel values and labels for the given index
        pixel_values = self.pixel_values[idx]
        label = self.labels[idx]

        # Return them as tensors
        return {
            'pixel_values': pixel_values,
            'labels': label,
        }

num_val_samples = 1000
num_test_samples = 1000

# # Calculate the number of samples to use for the validation set (10% of the total samples)
# num_val_samples = int(0.1 * len(data_dict['pixel_values']))
# num_test_samples = int(0.1 * len(data_dict['pixel_values']))


# Split the data_dict into training, validation, and test sets
train_data_dict = {
    'pixel_values': data_dict['pixel_values'][0:10986-1000-1000],
    'labels': data_dict['labels'][0:10986-1000-1000]
}


val_data_dict = {
    'pixel_values': data_dict['pixel_values'][10986-1000-1000:10986-1000],
    'labels': data_dict['labels'][10986-1000-1000:10986-1000]
}

test_data_dict = {
    'pixel_values': data_dict['pixel_values'][10986-1000:],
    'labels': data_dict['labels'][10986-1000:]
}


# Create training and validation datasets
training_dataset = CustomImageDataset(train_data_dict)
val_dataset = CustomImageDataset(val_data_dict)
test_dataset = CustomImageDataset(test_data_dict)


from transformers import Trainer
from transformers import TrainingArguments
# Training arguments
training_args = TrainingArguments(
    output_dir="./vit-base-beans-demo-v5",
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=0.0002,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)


# DataCollator for batch processing
class DataCollator:
    def __call__(self, batch):
        pixel_values = torch.stack([item['pixel_values'] for item in batch])
        labels = torch.tensor([item['labels'] for item in batch])
        return {
            'pixel_values': pixel_values,
            'labels': labels,
        }

data_collator = DataCollator()


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=training_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# Predict labels on the test dataset
predictions = trainer.predict(test_dataset)
# print(predictions.predictions)

# Get the predicted label IDs
predicted_label_ids = predictions.predictions.argmax(-1)

# Get the label names from the ID to label mapping
id2label = {str(i): label for i, label in enumerate(labels)}
predicted_labels = [id2label[str(label_id)] for label_id in predicted_label_ids]

predicted_labels

# Get the ground truth labels
true_labels = [id2label[str(label.item())] for label in test_dataset[:]['labels']]
true_labels

# Generate the classification report
from sklearn.metrics import classification_report
report = classification_report(true_labels, predicted_labels)
print(report)


def export_classification_report(report, mod, eval_set=None):
    lines = report.split('\n')
    data = lines[2:4] + lines[5:9]
    data = [line.split() for line in data]

    acc = float(data[2][1])
    m_prec = float(data[3][2])
    m_recall = float(data[3][3])
    m_f1 = float(data[3][4])
    w_prec = float(data[4][2])
    w_recall = float(data[4][3])
    w_f1 = float(data[4][4])

    df = pd.DataFrame({
        'Model': [mod],
        'EvalSet': [eval_set],
        'Accuracy': [acc],
        'M-Precision': [m_prec],
        'M-Recall': [m_recall],
        'M-F1-Score': [m_f1],
        'W-Precision': [w_prec],
        'W-Recall': [w_recall],
        'W-F1-Score': [w_f1],
    }) 

    return df


modname = 'Vit_MAMI_Imgs'

results_val= export_classification_report(report, modname, eval_set='test')
PERFORMANCE_RESULTS = pd.concat([PERFORMANCE_RESULTS, results_val])

del train_data_dict, val_data_dict, test_data_dict, total_img_info, total_image_info_tf 
del filtered_txt_file



torch.Size([10986, 3, 224, 224]) torch.Size([10986])
Model Configuration


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using cuda_amp half precision ba

In [ ]:
PERFORMANCE_RESULTS

In [ ]:
# PERFORMANCE_RESULTS.to_excel('16_classification_report_Model_ViT_MAMI_Imgs_result3_final.xlsx', 
#                       index=False)